<h1 style='font-size:40px'> HTTPS Setup with Kubernetes</h1>

<h2 style='font-size:30px'> 
    HTTPS Setup Overview
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Para habilitarmos conexões HTTPS em nossa aplicação, nosso site deverá ter um certificado que habilite o uso desse protocolo. Usaremos o serviço LetsEncrypt para obtê-lo. Ele é um Automatic Certificate Management Environment (ACME) gratuito.
        </li>
        <li>
            De maneira geral, quando invocarmos o LetsEncrypt, este fará uma requisição a um endereço específico do nosso site. Caso esta seja bem sucedida, receberemos o certificado.
        </li>
    </ul>
 </div>

<figure>
        <center style='margin-top:20px'>
            <img src='img/17_https_flow.png'>
                <figcaption> Ilustração do ciclo de obtenção do certificado.</figcaption>
        </center>
    </figure>

<h2 style='font-size:30px'> 
    Domain Purchase
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Aula em que compramos um nome de domínio para o nosso site. Em seguida, configuramos o site para apontar para o IP de nosso cluster.
        </li>
    </ul>
 </div>

<h2 style='font-size:30px'> 
    How to Wire Up Cert Manager
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            O fluxo de gerenciamento de certificados envolve três objetos do Kubernetes.
        </li>
        <li>
            O primeiro seria o Cert Manager, instalado com o Helm. Ele seria o responsável por se comunicar com a API do LetsEncrypt, obter o certificado e armazená-lo num objeto Secret.
        </li>
        <li>
            O segundo seria o Issuer, que basicamente informa ao CertManager o onde deverá solicitar o certificado. Já o último, o Certificate, conterá dados que o certificado deverá carregar.
        </li>
    </ul>
 </div>

<figure>
        <center style='margin-top:20px'>
            <img src='img/17_certificate_objects.png'>
        </center>
    </figure>

```sh
helm repo add jetstack https://charts.jetstack.io
helm repo update
helm install \
  cert-manager jetstack/cert-manager \
  --namespace cert-manager \
  --create-namespace \
  --version v1.8.0 \
  --set installCRDs=true
```
Comando usado para a instalação do CertManager no GKE.

<h2 style='font-size:30px'> 
    Issuer Config File
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Aula em que implantamos o arquivo de configuração de nosso Issuer.
        </li>
        <li>
            Sua estrutura se assemelha bastante com a de outros arquivos já feitos, com exceção de dois campos:
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> acme</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Campo em que declaramos o servidor do ACME a ser utilizado, junto com um email para identificação. 
        </li>
        <li>
            Além disso, declaramos um objeto Secret onde a chave privada de conexão ao ACME será armazenada. 
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> solvers</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Lista as metodologias de desafios que o ACME deverá realizar em nossa plataforma. No caso, o serviço criará um Ingress Service dentro do qual o desafio será conduzido.
        </li>
        <li>
            Deve-se ressaltar que o tipo de Ingress do desafio deverá ser o mesmo do da aplicação real.
        </li>
    </ul>
 </div>

```yaml
apiVersion: cert-manager.io/v1
kind: ClusterIssuer
metadata:
  name: letsencrypt-prod
spec:
  acme:
    server: https://acme-v02.api.letsencrypt.org/directory
    email: "user-email@email.com"
    privateKeySecretRef:
      name: letsencrypt-prod
    solvers:
      - http01:
          ingress:
            class: nginx
```
Exemplo de um .yaml de um Issuer.

<h2 style='font-size:30px'> 
    Certificate Config File
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Aula em que configuramos o .yaml do objeto Certificate. 
        </li>
        <li>
            Em sua <i> spec</i>, devemos declarar três campos essenciais para a criação da entidade.
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> secretName</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Nome do objeto Secret onde o certificado será armazenado.
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> issuerRef</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Referências ao objeto Issuer, que contém os parâmetros para o desafio do ACME.
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> commonName</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Nome interpretável a humanos que identifique o seu site. Recomenda-se aquele que a sua plataforma é mais conhecida. 
        </li>
    </ul>
 </div>

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> dnsNames</h4>
<div> 
    <ul style='font-size:20px'> 
        <li>
            NListagem com os nomes DNS que o certificado garantirá.
        </li>
    </ul>
 </div>

```yaml
apiVersion: cert-manager.io/v1
 
kind: Certificate
metadata:
  name: yourdomain-com-tls
spec:
  secretName: yourdomain-com
  issuerRef:
    name: letsencrypt-prod
    kind: ClusterIssuer
  commonName: yourdomain.com
  dnsNames:
    - yourdomain.com
    - www.yourdomain.com
```
Exemplo de um .yaml para um objeto Certificate.

In [6]:
! git add .
! git commit -m 'Aula 201'
! git push

[master 7da6104]  Explicar o campo http01 do yaml do Issuer
 2 files changed, 30 insertions(+), 28 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 24 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 779 bytes | 779.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/felipesveiga/docker.git
   7d35632..7da6104  master -> master


<p style='color:red'> Vi Aula 299 e 300; Aula 301</p>

<p style='color:red'> Visite a Aula 307, caso deseje deletar o seu cluster!</p>